In [1]:
import os, glob, re, io, random, gensim, smart_open, logging
import numpy as np

from PyPDF2 import PdfReader
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
from nltk.tokenize import word_tokenize


In [2]:
pdfReaders = []
pdfFiles = []
docLabels = []

# taggedData = []
# wordSets = []

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
rootDir = "/Users/tillman/t-root/dev/projects/2022/pdf-correlator/gitignored"
txtExtractDir = "/Users/tillman/t-root/dev/projects/2022/pdf-correlator/gitignored/txt-extractions/"

stoplist = set('for a of the and to in'.split(' '))

In [3]:
# read files
def read_files():
    os.chdir(rootDir)
    for file in glob.glob("**/*.pdf"):
        pdfFiles.append(file)
        pdfReaders.append(PdfReader(file))
    print("pdf files read")
            
read_files()

pdf files read


In [4]:
print(pdfReaders)

[<PyPDF2._reader.PdfReader object at 0x105782aa0>, <PyPDF2._reader.PdfReader object at 0x1057a2530>]


In [22]:
def extract_to_txt():
    os.chdir(txtExtractDir)
    pat0 = ('(?<!Dr)(?<!Esq)\. +(?=[A-Z])')
    pat1 = ('\.+(?=[A-Z])')
    pat2 = ('\.+(?=[0-9])')
    pat3 = ('\. +(?=[0-9])')
    pat4 = ('(?=[for a of the and to in])')
    
    patterns = [pat0, pat1, pat2, pat3, pat4]
    for i in pdfReaders:
        with open(i.metadata.title + ".txt", 'w', encoding="utf-8") as file:
            
            # add doc title to array for reference / tagging
            docLabels.append(i.metadata.title)
            
            for j in range(0, len(i.pages)):
                # format txt file so that each line is one sentence (doc2vec requirement)
                text = i.getPage(j).extract_text()
                text = re.sub(patterns[0], '.\n', text)
                text = re.sub(patterns[1], '.\n', text)
                text = re.sub(patterns[2], '.\n', text)
                text = re.sub(patterns[3], '.\n', text)
                text = re.sub(patterns[4], '', text)
                file.write(text)    

extract_to_txt()
print("extracted files labeled: " + str(docLabels))



extracted files labeled: ['new-dark-age', 'g5grap.lo', 'new-dark-age', 'new-dark-age', 'g5grap.lo', 'new-dark-age', 'g5grap.lo', 'new-dark-age', 'new-dark-age', 'new-dark-age', 'new-dark-age', 'g5grap.lo', 'new-dark-age', 'g5grap.lo']


In [35]:
class CorpusGen(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self, tokens_only=False):
        for fname in os.listdir(self.dirname):
            with smart_open.open(fname, encoding="iso-8859-1") as f:
                for i, line in enumerate(f):
                    tokens = gensim.utils.simple_preprocess(line, min_len=2, max_len=15, deacc=True)
                    if tokens_only:
                        yield tokens
                    else:
                        yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

trainCorpus = list(CorpusGen('/Users/tillman/t-root/dev/projects/2022/pdf-correlator/gitignored/txt-extractions'))


print(list(trainCorpus))

[TaggedDocument(words=['new', 'dark', 'agethis', 'ebook', 'is', 'licensed', 'to', 'lena', 'gieseke', 'lenagieseke', 'gmail', 'com', 'on', 'new', 'dark', 'the', 'end', 'of', 'the', 'futurejames', 'bridle'], tags=[0]), TaggedDocument(words=['this', 'ebook', 'is', 'licensed', 'to', 'lena', 'gieseke', 'lenagieseke', 'gmail', 'com', 'on', 'this', 'paperback', 'edition', 'first', 'published', 'by', 'verso', 'first', 'published', 'by', 'verso', 'james', 'bridle', 'all', 'rights', 'reservedthe', 'moral', 'rights', 'of', 'the', 'author', 'have', 'been', 'asserted', 'versouk', 'meard', 'street', 'london', 'egus', 'jay', 'street', 'suite', 'brooklyn', 'ny', 'versobooks', 'comverso', 'is', 'the', 'imprint', 'of', 'new', 'left', 'booksisbn', 'isbn', 'uk', 'ebk', 'isbn', 'us', 'ebk', 'british', 'library', 'cataloguing', 'in', 'publication', 'dataa', 'catalogue', 'record', 'for', 'this', 'book', 'isavailable', 'from', 'the', 'british', 'librarylibrary', 'of', 'congress', 'cataloging', 'in', 'publicat

In [ ]:
# class MyCorpus:
#     def __iter__(self):
#         for line in open('/Users/tillman/t-root/dev/projects/2022/pdf-correlator/gitignored/txt-extractions/new-dark-age.txt'):
#             # assume there's one document per line, tokens separated by whitespace
#             yield dictionary.doc2bow(line.lower().split())
                
# trainCorpus = MyCorpus()

# print(list(trainCorpus))
    

In [ ]:
# def read_corpus(fname, tokens_only=False):
        
#     with smart_open.open(fname, encoding="iso-8859-1") as f:
#         for i, line in enumerate(f):
#             tokens = gensim.utils.simple_preprocess(line)
#             if tokens_only:
#                 yield tokens
#             else:
#                 yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

In [ ]:
# with open(txtExtractDir + 'fullcorpus.txt', 'w+') as file:
#     for items in sentences:
#         file.writelines('%s\n' %items)

In [ ]:
# trainCorpus = list(read_corpus('/Users/tillman/t-root/dev/projects/2022/pdf-correlator/gitignored/txt-extractions/new-dark-age.txt'))
    
# print(trainCorpus)

In [37]:
# train the model

model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(trainCorpus)


2022-09-23 20:23:48,089 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d50,n5,w5,mc2,s0.001,t3>', 'datetime': '2022-09-23T20:23:48.089729', 'gensim': '4.2.0', 'python': '3.10.6 | packaged by conda-forge | (main, Aug 22 2022, 20:41:22) [Clang 13.0.1 ]', 'platform': 'macOS-12.0.1-arm64-arm-64bit', 'event': 'created'}
2022-09-23 20:23:48,092 : INFO : collecting all words and their counts
2022-09-23 20:23:48,094 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2022-09-23 20:23:48,117 : INFO : collected 15641 word types and 3552 unique tags from a corpus of 3812 examples and 78890 words
2022-09-23 20:23:48,118 : INFO : Creating a fresh vocabulary
2022-09-23 20:23:48,133 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=2 retains 6032 unique words (38.57% of original 15641, drops 9609)', 'datetime': '2022-09-23T20:23:48.133483', 'gensim': '4.2.0', 'python': '3.10.6 | packaged by conda-forge | (main, Aug 22 2022, 20:41:22) [Clang 13.

In [38]:
# check word occurence
print(f"Word 'the' appeared {model.wv.get_vecattr('the', 'count')} times in the training corpus.")

Word 'the' appeared 5001 times in the training corpus.


In [39]:
model.train(trainCorpus, total_examples=model.corpus_count, epochs=model.epochs)

2022-09-23 20:23:54,727 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 3 workers on 6032 vocabulary and 50 features, using sg=0 hs=0 sample=0.001 negative=5 window=5 shrink_windows=True', 'datetime': '2022-09-23T20:23:54.727481', 'gensim': '4.2.0', 'python': '3.10.6 | packaged by conda-forge | (main, Aug 22 2022, 20:41:22) [Clang 13.0.1 ]', 'platform': 'macOS-12.0.1-arm64-arm-64bit', 'event': 'train'}
2022-09-23 20:23:54,839 : INFO : EPOCH 0: training on 78890 raw words (57010 effective words) took 0.1s, 555879 effective words/s
2022-09-23 20:23:54,932 : INFO : EPOCH 1: training on 78890 raw words (56964 effective words) took 0.1s, 625438 effective words/s
2022-09-23 20:23:55,026 : INFO : EPOCH 2: training on 78890 raw words (57022 effective words) took 0.1s, 620904 effective words/s
2022-09-23 20:23:55,117 : INFO : EPOCH 3: training on 78890 raw words (56951 effective words) took 0.1s, 639939 effective words/s
2022-09-23 20:23:55,212 : INFO : EPOCH 4: training on 78890 

In [40]:
# infer a vector from corupus (I dont actually know what this means or does! :D )
vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
print("infering a vector:")
print(vector)

infering a vector:
[-0.03381298 -0.09673154 -0.08867207 -0.02666846  0.00207447 -0.12913707
  0.05895968  0.05944204 -0.08860318  0.01033913 -0.0645445   0.15196165
  0.02392953  0.06114365  0.17482369  0.07791467 -0.04676589 -0.14756292
 -0.06303622 -0.24206112  0.05551238 -0.16148023 -0.08604905 -0.00847116
  0.19520094 -0.02415288  0.03707894 -0.20077267 -0.14620225  0.08362661
  0.01013568  0.0100645   0.0633244   0.00575125  0.02024393  0.32893497
  0.13864346 -0.01221491  0.14832157 -0.11295217  0.16493703  0.09097979
  0.11819393  0.04164441 -0.01995114 -0.15335572  0.05710175 -0.10241127
 -0.03709977  0.00533394]


In [41]:
print(list(trainCorpus))

[TaggedDocument(words=['new', 'dark', 'agethis', 'ebook', 'is', 'licensed', 'to', 'lena', 'gieseke', 'lenagieseke', 'gmail', 'com', 'on', 'new', 'dark', 'the', 'end', 'of', 'the', 'futurejames', 'bridle'], tags=[0]), TaggedDocument(words=['this', 'ebook', 'is', 'licensed', 'to', 'lena', 'gieseke', 'lenagieseke', 'gmail', 'com', 'on', 'this', 'paperback', 'edition', 'first', 'published', 'by', 'verso', 'first', 'published', 'by', 'verso', 'james', 'bridle', 'all', 'rights', 'reservedthe', 'moral', 'rights', 'of', 'the', 'author', 'have', 'been', 'asserted', 'versouk', 'meard', 'street', 'london', 'egus', 'jay', 'street', 'suite', 'brooklyn', 'ny', 'versobooks', 'comverso', 'is', 'the', 'imprint', 'of', 'new', 'left', 'booksisbn', 'isbn', 'uk', 'ebk', 'isbn', 'us', 'ebk', 'british', 'library', 'cataloguing', 'in', 'publication', 'dataa', 'catalogue', 'record', 'for', 'this', 'book', 'isavailable', 'from', 'the', 'british', 'librarylibrary', 'of', 'congress', 'cataloging', 'in', 'publicat

In [42]:
# assessing the model
ranks = []
secondRanks = []
for doc_id in range(len(trainCorpus)):
        inferredVector = model.infer_vector(trainCorpus[doc_id].words)
        sims = model.dv.most_similar([inferredVector], topn=len(model.dv))
        rank = [docid for docid, sim in sims].index(doc_id)
        ranks.append(rank)
        print(doc_id)
        
        secondRanks.append(sims[1])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098


3549
3550
3551


ValueError: 3552 is not in list

In [44]:
import collections

counter = collections.Counter(ranks)
print(counter)


Counter({0: 2880, 1: 78, 2: 31, 7: 16, 3: 16, 4: 9, 10: 8, 6: 7, 5: 6, 8: 6, 30: 5, 11: 4, 16: 4, 28: 4, 33: 4, 18: 4, 22: 4, 24: 3, 12: 3, 49: 3, 40: 3, 136: 3, 14: 3, 20: 3, 25: 3, 3371: 3, 23: 3, 15: 3, 3522: 3, 45: 2, 41: 2, 19: 2, 31: 2, 134: 2, 36: 2, 32: 2, 27: 2, 51: 2, 60: 2, 13: 2, 68: 2, 196: 2, 3160: 2, 337: 2, 1902: 2, 3544: 2, 305: 2, 2930: 2, 580: 2, 188: 2, 1609: 2, 34: 2, 3502: 2, 54: 2, 17: 2, 3532: 2, 148: 2, 154: 2, 3530: 2, 3537: 2, 1348: 2, 2704: 2, 1002: 2, 632: 2, 3003: 2, 3433: 2, 2680: 2, 1538: 2, 26: 2, 67: 2, 3545: 2, 138: 1, 919: 1, 1359: 1, 1586: 1, 346: 1, 880: 1, 604: 1, 1217: 1, 2971: 1, 845: 1, 202: 1, 1254: 1, 1043: 1, 105: 1, 612: 1, 97: 1, 1163: 1, 21: 1, 100: 1, 684: 1, 410: 1, 889: 1, 76: 1, 511: 1, 77: 1, 1770: 1, 801: 1, 905: 1, 411: 1, 2850: 1, 320: 1, 253: 1, 3449: 1, 761: 1, 224: 1, 1636: 1, 2129: 1, 1353: 1, 46: 1, 89: 1, 738: 1, 953: 1, 1124: 1, 2097: 1, 244: 1, 199: 1, 628: 1, 1661: 1, 2316: 1, 3235: 1, 3065: 1, 352: 1, 399: 1, 301: 1, 347

In [45]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(trainCorpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(trainCorpus[sims[index][0]].words)))

Document (3552): «september october published by the ieee computer society»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec<dm/m,d50,n5,w5,mc2,s0.001,t3>:

MOST (3353, 0.8487202525138855): «jennifer king deirdre mulligan and stephen rafael citris report the san safety programa uc berkeley december available at wired com»

SECOND-MOST (3361, 0.8437795639038086): «for good overview of the trial see kevin hall the abc trial originally published atukcoldwar simplenet com archived at archive li xft»

MEDIAN (670, 0.4687787890434265): «outside of the scope of the study forexample are power lines despite the essential nature of their services toict the council for science and technologya study on the other hand notes that one of the limiting factors for the transfer of electricity byoverhead transmission lines is their thermal capacity which is affected bythe ambient air temperature»

LEAST (3131, -0.3792306184768677): «»

